In [1]:
from __future__ import annotations
from dataclasses import dataclass
from typing import List, Annotated

from delierium.JanetBasis import _Dterm
from delierium.MatrixOrder import Context
from delierium.helpers import is_derivative, is_function, order_of_derivative
from IPython.core.debugger import set_trace

In [2]:
@dataclass
class _Dterm1:
    coefficient: Expression = 1
    derivative: sage.modules.vector_integer_dense.Vector_integer_dense | None = None
    context: Context | None = None
    _order:sage.modules.vector_integer_dense.Vector_integer_dense | None = None
    
    def __post_init__(self):
        if isinstance(self.derivative, Expression):
            self.derivative = vector(order_of_derivative(self.derivative))
        elif self.derivative is None:
            self.derivative = vector([0] * len(self.context._independent))
        elif isinstance(self.derivative, type([])):
            self.derivative = vector(self.derivative)
        
            
    @classmethod
    def from_expression(cls, expr: Expression, context: Context) -> _Dterm1:
        d = None
        c = 1
        if is_derivative(expr) or is_function(expr):
            d = order_of_derivative(expr)
        else:
            for o in expr.operands():
                if is_derivative(o) or is_function(o):
                    d = order_of_derivative(o)
                else:
                    c *= o
        return cls(coefficient=c, derivative=d, context=context)

    @classmethod
    def _compute_order(cls, expr: Expression, context:Context) -> sage.modules.vector_integer_dense.Vector_integer_dense:
        """computes the monomial tuple from the derivative part"""
        if is_derivative(expr):
            return order_of_derivative(expr, len(context._independent))
        else:
            return [0] * len(context._independent)
        
    def diff(self, *vars) -> Annotated[list[_Dterm1]]:
        """applies differentiation to _Dterm1. Vars is either a list
        of variables, or a vector of indices
        """
        if isinstance(vars[0], sage.rings.integer.Integer):
            # got an vector of indices
            index_vector = vars
            real_vars = [self.context._independent.index(_) for _ in vars]
        else:
            index_vector = [0]*len(self.context._independent)
            for v in vars:
                index_vector[self.context._independent.index(v)] += 1
            real_vars    = vars
        if isinstance(self.coefficient, sage.rings.integer.Integer):
            return [_Dterm1(derivative=self.derivative + vector(index_vector), context=self.context)]
        if self.derivative is None:
            return [_Dterm1(coefficient=self.coefficient.diff(*real_vars), context=self.context)]
        return [_Dterm1(derivative=self.derivative + vector(index_vector), coefficient=self.coefficient, context=self.context),
                _Dterm1(derivative=self.derivative, coefficient=self.coefficient.diff(*real_vars), context=self.context)]

In [3]:
x,y=var('x y')
phi=function('phi', latex_name=r"\phi")
xi=function('xi', latex_name=r"\xi")

ctx = Context([xi, phi], [y, x])
d2 = _Dterm1(derivative=diff(phi(x,y), x,x,y), context=ctx)

In [4]:
print(d2)

_Dterm1(coefficient=1, derivative=(2, 1), context=<delierium.MatrixOrder.Context object at 0x7f3fa82d5740>, _order=None)


In [5]:
print(d2.diff(x,x,x))

> /tmp/ipykernel_29431/3189818986.py(54)diff()
     52         from IPython.core.debugger import set_trace
     53         set_trace()
---> 54         if isinstance(self.coefficient, sage.rings.integer.Integer):
     55             return [_Dterm1(derivative=self.derivative + vector(index_vector), context=self.context)]
     56         if self.derivative is None:



ipdb>  c


[_Dterm1(coefficient=1, derivative=(2, 4), context=<delierium.MatrixOrder.Context object at 0x7f3fa82d5740>, _order=None)]


In [6]:
#import doctest
#doctest.testmod()
d3=_Dterm1(x*y**2, context=ctx)

In [7]:
print(d3)

_Dterm1(coefficient=x*y^2, derivative=(0, 0), context=<delierium.MatrixOrder.Context object at 0x7f3fa82d5740>, _order=None)


In [8]:
d3.diff(x)

> /tmp/ipykernel_29431/3189818986.py(54)diff()
     52         from IPython.core.debugger import set_trace
     53         set_trace()
---> 54         if isinstance(self.coefficient, sage.rings.integer.Integer):
     55             return [_Dterm1(derivative=self.derivative + vector(index_vector), context=self.context)]
     56         if self.derivative is None:



ipdb>  c


[_Dterm1(coefficient=x*y^2, derivative=(0, 1), context=<delierium.MatrixOrder.Context object at 0x7f3fa82d5740>, _order=None),
 _Dterm1(coefficient=y^2, derivative=(0, 0), context=<delierium.MatrixOrder.Context object at 0x7f3fa82d5740>, _order=None)]

In [9]:
d3=_Dterm1.from_expression(x*y**2*diff(xi(x,y), x,x), context=ctx)

In [10]:
print(d3)

_Dterm1(coefficient=x*y^2, derivative=(2, 0), context=<delierium.MatrixOrder.Context object at 0x7f3fa82d5740>, _order=None)


In [11]:
d4=_Dterm1.from_expression(diff(phi(y,x),x,x)*x*y**2, context=ctx); print(d4)

_Dterm1(coefficient=x*y^2, derivative=(0, 2), context=<delierium.MatrixOrder.Context object at 0x7f3fa82d5740>, _order=None)


In [12]:
print(d4.diff(x,x,x))

> /tmp/ipykernel_29431/3189818986.py(54)diff()
     52         from IPython.core.debugger import set_trace
     53         set_trace()
---> 54         if isinstance(self.coefficient, sage.rings.integer.Integer):
     55             return [_Dterm1(derivative=self.derivative + vector(index_vector), context=self.context)]
     56         if self.derivative is None:



ipdb>  c


[_Dterm1(coefficient=x*y^2, derivative=(0, 5), context=<delierium.MatrixOrder.Context object at 0x7f3fa82d5740>, _order=None), _Dterm1(coefficient=0, derivative=(0, 2), context=<delierium.MatrixOrder.Context object at 0x7f3fa82d5740>, _order=None)]
